### Review : Let's go deep & wide

In [11]:
import tensorflow as tf

In [2]:
# RV in uniform distribution, range = -1.0 ~ 1.0
W1 = tf.Variable(tf.random_uniform([2, 5], -1.0, 1.0)) # 처음 input인 2가 중요(정해진 숫자), 5는 내가 마음대로 선정한 숫자
# 윗줄(W1=1번째 layer)의 output값은 5개 이므로, 이번 layer의 input은 5가 되고, output은 4로 정함
W2 = tf.Variable(tf.random_uniform([5, 4], -1.0, 1.0)) 
W3 = tf.Variable(tf.random_uniform([4, 1], -1.0, 1.0)) # 최종 output은 0아니면 1이므로 1개만 출력한다.

b1 = tf.Variable(tf.zeros([5]), name='bias1')
b2 = tf.Variable(tf.zeros([4]), name='bias2')
b3 = tf.Variable(tf.zeros([1]), name='bias3')

In [4]:
# # set hypothesis
L2 = tf.sigmoid(tf.matmul(X,W1) + b1)
L3 = tf.sigmoid(tf.matmul(L2, W2) + b2)
hypothesis = tf.sigmoid(tf.matmul(L3, W3) + b3)

##### Layer가 9단으로 늘리면, Layer가 1개인 결과와 같이 poor result도출한다....why?
- Backpropagation(chain rule)에 의해 미분값을 뒤에서 부터 앞으로 곱해나갔었다.
- 만약 df/dx = y인 경우, y가 0인 경우가 나타날 수 있다.
- 즉, y보다 앞의 layer에서 sigmoid input값이 -10정도여서 sigmoid output이 0에 가깝게 나온 경우를 말한다.
- 결국, backpropagation시 가장 앞단 input으로 최종output을 미분한 값이 0이 된다.(영향력=gradient=0)
- 이를 Vanishing Gradient problem이 발생한다.

##### Solution : Sigmoid? Nope!....ReLU(Rectified Linear Unit)!!
- 0보다 작은 값을 곱하게 하는 sigmoid대신 ReLU 사용하자
- 0보다 작을 땐, 0이지만, 0보다 큰 input에 대해선 input값에 비례해 output값을 한없이 크게 만든다.
- 0보다 작은 input에 0이 아닌 -0.1xxxx값을 주는 것이 Leaky ReLU
- `tf.nn.relu(tf.matmul(X,W1) + b1)`

### DeepLearning with MNIST
- review Lab 7 
- Neural Net

In [3]:
# review Lab 7 Learning rate and Evaluation
import tensorflow as tf
import random
# import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# weights & bias for nn layers
W = tf.Variable(tf.random_normal([784, 10]))
b = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels}))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Epoch: 0001 cost = 7.165734003
Epoch: 0002 cost = 1.852857486
Epoch: 0003 cost = 1.169201553
Epoch: 0004 cost = 0.917270895
Epoch: 0005 cost = 0.779720637
Epoch: 0006 cost = 0.690264096
Epoch: 0007 cost = 0.625711640
Epoch: 0008 cost = 0.577925111
Epoch: 0009 cost = 0.540020572
Epoch: 0010 cost = 0.5

#### Neural Net - MLP

In [10]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

In [12]:
# set weights & bias for NN Layers
W1 = tf.Variable(tf.random_normal([784, 256]), name="weight1")
b1 = tf.Variable(tf.random_normal([256]), name='bias1')
L1 = tf.nn.relu(tf.matmul(X,W1) + b1)

W2 = tf.Variable(tf.random_normal([256, 256]), name="weight2")
b2 = tf.Variable(tf.random_normal([256]), name='bias2')
L2 = tf.nn.relu(tf.matmul(L1,W2) + b2)

W3 = tf.Variable(tf.random_normal([256, 10]), name="weight3")
b3 = tf.Variable(tf.random_normal([10]), name='bias3')
logits = tf.matmul(L2,W3) + b3

hypothesis = tf.nn.softmax(logits)

In [13]:
# set cost function & minimize
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [17]:
# initialize
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # train my model
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)

        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feed_dict = {X: batch_xs, Y: batch_ys}
            c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            avg_cost += c / total_batch

        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

    print('Learning Finished!')

    # Test model and check accuracy
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('Accuracy:', sess.run(accuracy, feed_dict={
          X: mnist.test.images, Y: mnist.test.labels}))

Epoch: 0001 cost = 183.817682183
Epoch: 0002 cost = 45.162245911
Epoch: 0003 cost = 28.341257000
Epoch: 0004 cost = 19.891212168
Epoch: 0005 cost = 14.521632847
Epoch: 0006 cost = 10.849469518
Epoch: 0007 cost = 8.188141153
Epoch: 0008 cost = 6.141151408
Epoch: 0009 cost = 4.662250947
Epoch: 0010 cost = 3.600411860
Epoch: 0011 cost = 2.669602572
Epoch: 0012 cost = 2.114164479
Epoch: 0013 cost = 1.695043403
Epoch: 0014 cost = 1.302944874
Epoch: 0015 cost = 0.947845555
Learning Finished!
Accuracy: 0.9453


#### Xavier for MNIST

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [16]:
# weight & bias for NN Layers, Xavier initializer
W1 = tf.get_variable("xavier_w1", shape=[784, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("xavier_w2", shape=[256, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("xavier_w3", shape=[256, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

In [17]:
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [18]:
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels}))


Epoch: 0001 cost = 0.298402226
Epoch: 0002 cost = 0.116010743
Epoch: 0003 cost = 0.078538435
Epoch: 0004 cost = 0.057424470
Epoch: 0005 cost = 0.042347964
Epoch: 0006 cost = 0.031848474
Epoch: 0007 cost = 0.025207561
Epoch: 0008 cost = 0.019937467
Epoch: 0009 cost = 0.019290161
Epoch: 0010 cost = 0.014684581
Epoch: 0011 cost = 0.012416943
Epoch: 0012 cost = 0.011082378
Epoch: 0013 cost = 0.012820295
Epoch: 0014 cost = 0.009086280
Epoch: 0015 cost = 0.012340207
Learning Finished!
Accuracy: 0.9787


epoch=1에서 부터 cost가 0.29xxx로 굉장히 낮다.

#### Deep & Wide NN
- layer의 갯수와 output의 갯수를 늘려보자

In [3]:
# weight & bias for NN Layers, Xavier initializer
W1 = tf.get_variable("W1", shape=[784,512], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X,W1) + b1)

W2 = tf.get_variable("W2", shape=[512,512], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1,W2) + b2)

W3 = tf.get_variable("W3", shape=[512,512], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3= tf.nn.relu(tf.matmul(L2, W3) + b3)

W4 = tf.get_variable("W4", shape=[512,512], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]))
L4= tf.nn.relu(tf.matmul(L3, W4) + b4)

W5 = tf.get_variable("W5", shape=[512,10], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis= tf.matmul(L4, W5) + b5

In [19]:
# set cost function & optimize, minimize
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [20]:
# training!!
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # train my model
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)

        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feed_dict = {X: batch_xs, Y: batch_ys}
            c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            avg_cost += c / total_batch

        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

    print('Learning Finished!')

    # Test model and check accuracy
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('Accuracy:', sess.run(accuracy, feed_dict={
          X: mnist.test.images, Y: mnist.test.labels}))

Epoch: 0001 cost = 0.288647405
Epoch: 0002 cost = 0.113488720
Epoch: 0003 cost = 0.077498346
Epoch: 0004 cost = 0.052882102
Epoch: 0005 cost = 0.042813483
Epoch: 0006 cost = 0.031092437
Epoch: 0007 cost = 0.024971649
Epoch: 0008 cost = 0.019067265
Epoch: 0009 cost = 0.018711744
Epoch: 0010 cost = 0.017897934
Epoch: 0011 cost = 0.012748012
Epoch: 0012 cost = 0.009399581
Epoch: 0013 cost = 0.015755477
Epoch: 0014 cost = 0.009698919
Epoch: 0015 cost = 0.007765776
Learning Finished!
Accuracy: 0.9761


오히려 accuracy가 떨어짐..overfitting!!

#### Dropout for MNIST
- Dropout Layer를 추가해준다.
- train할 때는 keep_prob=0.5~0.7로 주지만, test시에는 keep_prob=1 (중요)!!!

In [7]:
# Lab 10 MNIST and Dropout
import tensorflow as tf
import random
# import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
# dropout (keep_prob) rate  0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# weights & bias for nn layers
# http://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
W1 = tf.get_variable("dropout_W1", shape=[784, 512],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("dropout_W2", shape=[512, 512],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("dropout_W3", shape=[512, 512],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("dropout_W4", shape=[512, 512],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.get_variable("dropout_W5", shape=[512, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

In [10]:
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
for epoch in range(training_epochs):
    avg_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Epoch: 0001 cost = 0.458002190
Epoch: 0002 cost = 0.174828656
Epoch: 0003 cost = 0.129483507
Epoch: 0004 cost = 0.105185423
Epoch: 0005 cost = 0.087631562
Epoch: 0006 cost = 0.081366653
Epoch: 0007 cost = 0.074339970
Epoch: 0008 cost = 0.067223198
Epoch: 0009 cost = 0.062889992
Epoch: 0010 cost = 0.059598179
Epoch: 0011 cost = 0.052880399
Epoch: 0012 cost = 0.054975953
Epoch: 0013 cost = 0.050065455
Epoch: 0014 cost = 0.048162811
Epoch: 0015 cost = 0.045250886
Learning Finished!
Accuracy: 0.9817
